In [4]:
'''
{
    "id": Value(dtype="string", id=None),
    "question": Value(dtype="string", id=None),
    "document_ids": [document_id, document_id, ...], -> max=100
}
'''

'\n{\n    "id": Value(dtype="string", id=None),\n    "question": Value(dtype="string", id=None),\n    "document_ids": [document_id, document_id, ...], -> max=100\n}\n'

In [5]:
import json
import pandas as pd
from datasets import load_from_disk
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer

import torch
from tqdm import tqdm
import numpy as np
import random



random.seed(42)

'''data'''
# data = json.load(open('data/raw/wikipedia_documents.json'))
data = json.load(open('../data/preprocessed/wikipedia_model.json'))
wiki = pd.DataFrame(data).T

dataset = load_from_disk("../data/raw/train_dataset/")
train_df = pd.DataFrame(dataset['train'])
valid_df = pd.DataFrame(dataset['validation'])
mrc = pd.concat([train_df, valid_df])

query_n = 5 # query number
wiki_n = 10 # wiki number
# query_n = 50 # query number
# wiki_n = 100 # wiki number

# wiki = random wiki + correct answer included
sampled_indices = random.sample(range(len(mrc)), query_n)
queries = [mrc['question'].iloc[idx] for idx in sampled_indices]
wiki_list = random.sample(wiki['text'].tolist(), wiki_n - query_n) + correct_contexts
random.shuffle(wiki_list)

'''model'''
prompts = {
    "query": "query: ",        # 검색 쿼리 프롬프트
    "passage": "passage: "     # 문서 패시지 프롬프트
}

model_name = "nlpai-lab/KoE5"
model = SentenceTransformer(
    model_name_or_path=model_name, 
    device='cuda', 
    # truncate_dim=512,
    # model_kwargs={"torch_dtype": torch.bfloat16},
    prompts=prompts,
    )
model.eval()

print(f"Queries n: {query_n}")
print(f"From wiki n: {wiki_n}")
print(f'Model: {model_name}')

NameError: name 'correct_contexts' is not defined

In [204]:
with torch.no_grad():
    query_embeddings = model.encode(queries, prompt_name="query", show_progress_bar=True)
    document_embeddings = model.encode(wiki_list, prompt_name="passage", show_progress_bar=True)
    similarities = model.similarity(query_embeddings, document_embeddings) * 100

Batches: 100%|██████████| 1/1 [00:00<00:00,  5.82it/s]


In [205]:
similarities

tensor([[ 25.5291,  15.3601,  48.1743,  -5.0732,  -8.0032,   9.1429],
        [ -1.5471,  -9.9796,  -2.5735,  -5.9626,  -4.6221,  27.8448],
        [  0.5115,  -7.0441,   6.2993,   5.7085,  -1.9781,   5.6421],
        [  8.9466,  19.1244,  19.4927,  -9.9744,  -7.0451,   5.8001],
        [ -4.5094, -11.2268,   2.2449,  -6.1181,   7.2080,   2.5567]])

In [206]:
similarities = similarities.cpu().numpy()
top_indices = np.argsort(similarities)[::-1][:]


In [207]:
top_indices

array([[1, 3, 0, 2, 5, 4],
       [3, 4, 5, 0, 1, 2],
       [1, 4, 0, 5, 3, 2],
       [1, 3, 4, 2, 0, 5],
       [4, 3, 5, 1, 0, 2]])

In [6]:
raw_data = json.load(open('../data/raw/wikipedia_documents.json'))
raw_wiki = pd.DataFrame(raw_data).T

model_data = json.load(open('../data/preprocessed/wikipedia_model.json'))
model_wiki = pd.DataFrame(model_data).T

dataset = load_from_disk("../data/raw/train_dataset/")
train_df = pd.DataFrame(dataset['train'])
valid_df = pd.DataFrame(dataset['validation'])
mrc = pd.concat([train_df, valid_df])

In [12]:
random.seed(42)

query_n = 40 # query number
wiki_n = 100 # wiki number

assert query_n <= wiki_n, 'query_n should be smaller than total wiki_n'

sampled_indices = random.sample(range(len(mrc)), query_n)
print(f'Sampled indices (Randomly Sampled 0~len(mrc)): {sampled_indices}')

mrc_samples = mrc.iloc[sampled_indices]
raw_wiki_document_ids = mrc_samples['document_id'].tolist()
model_wiki_samples = model_wiki[model_wiki['document_id'].isin(raw_wiki_document_ids)]
available_document_ids = [i for i in range(model_wiki['document_id'].max()) if i not in raw_wiki_document_ids]
extra_raw_wiki_document_ids = random.sample(available_document_ids, wiki_n - query_n)
print(f'Raw wiki document ids in MRC samples df: {raw_wiki_document_ids}')
print(f'Extra Raw wiki document ids: {extra_raw_wiki_document_ids}')
print(f'MRC samples df added: {mrc_samples.shape[0]}')
print(f'Model-Wiki samples df added (Answer): {model_wiki_samples.shape[0]}')

extra_model_wiki_samples = model_wiki[model_wiki['document_id'].isin(extra_raw_wiki_document_ids)]
model_wiki_samples = pd.concat([model_wiki_samples, extra_model_wiki_samples])
print(f'Model-Wiki samples df added (Extra): {extra_model_wiki_samples.shape[0]}')
print(f'Model-Wiki samples df added (Answer + Extra): {model_wiki_samples.shape[0]}')


Sampled indices (Randomly Sampled 0~len(mrc)): [912, 204, 2253, 2006, 1828, 1143, 839, 712, 3456, 260, 244, 767, 1791, 1905, 4139, 217, 1628, 3436, 1805, 3679, 2278, 53, 1307, 3462, 2787, 2276, 1273, 1763, 2757, 837, 759, 3112, 792, 2940, 2817, 2166, 355, 3763, 1022, 3100]
Raw wiki document ids in MRC samples df: [32341, 43424, 9619, 49578, 15171, 33522, 14801, 49158, 41569, 14104, 8480, 9492, 51644, 45759, 52838, 33762, 7778, 7833, 5661, 43755, 10243, 8118, 30148, 27570, 59022, 7560, 39694, 53806, 54884, 57067, 22849, 56126, 55698, 36895, 15825, 7693, 40757, 23431, 31749, 49856]
Extra Raw wiki document ids: [5164, 36200, 19227, 54390, 41223, 40559, 58067, 56510, 23716, 37860, 12611, 46203, 4558, 3003, 43362, 14947, 50695, 18979, 5229, 56091, 15269, 56823, 6620, 24927, 18231, 29731, 41686, 54699, 23925, 10669, 24276, 23298, 13740, 43948, 17510, 46023, 44824, 42495, 4679, 39944, 41639, 11225, 35027, 47813, 16057, 10718, 30312, 24883, 17705, 41969, 45127, 36522, 14403, 44894, 21266, 5527

In [8]:
import json
import pandas as pd
from datasets import load_from_disk
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer

import torch
from tqdm import tqdm
import numpy as np
import random



random.seed(42)

'''data'''
# data = json.load(open('data/raw/wikipedia_documents.json'))
model_data = json.load(open('../data/preprocessed/wikipedia_model.json'))
model_wiki = pd.DataFrame(model_data).T

dataset = load_from_disk("../data/raw/train_dataset/")
train_df = pd.DataFrame(dataset['train'])
valid_df = pd.DataFrame(dataset['validation'])
mrc = pd.concat([train_df, valid_df])

# query_n = 50 # query number
# wiki_n = 100 # wiki number
query_n = 5 # query number
wiki_n = 10 # wiki number


assert query_n <= wiki_n, 'query_n should be smaller than total wiki_n'

sampled_indices = random.sample(range(len(mrc)), query_n)
print(f'Sampled indices (Randomly Sampled 0~len(mrc)): {sampled_indices}')

mrc_samples = mrc.iloc[sampled_indices]
raw_wiki_document_ids = mrc_samples['document_id'].tolist()
model_wiki_samples = model_wiki[model_wiki['document_id'].isin(raw_wiki_document_ids)]
available_document_ids = [i for i in range(model_wiki['document_id'].max()) if i not in raw_wiki_document_ids]
extra_raw_wiki_document_ids = random.sample(available_document_ids, wiki_n - query_n)
print(f'Raw wiki document ids in MRC samples df: {raw_wiki_document_ids}')
print(f'Extra Raw wiki document ids: {extra_raw_wiki_document_ids}')
print(f'MRC samples df added: {mrc_samples.shape[0]}')
print(f'Model-Wiki samples df added (Answer): {model_wiki_samples.shape[0]}')

extra_model_wiki_samples = model_wiki[model_wiki['document_id'].isin(extra_raw_wiki_document_ids)]
model_wiki_samples = pd.concat([model_wiki_samples, extra_model_wiki_samples])
print(f'Model-Wiki samples df added (Extra): {extra_model_wiki_samples.shape[0]}')
print(f'Model-Wiki samples df added (Answer + Extra): {model_wiki_samples.shape[0]}')


'''model'''
prompts = {
    "query": "query: ",        # 검색 쿼리 프롬프트
    "passage": "passage: "     # 문서 패시지 프롬프트
}

model_name = "nlpai-lab/KoE5"
model = SentenceTransformer(
    model_name_or_path=model_name, 
    device='cuda', 
    similarity_fn_name='dot',
    # truncate_dim=512,
    # model_kwargs={"torch_dtype": torch.bfloat16},
    prompts=prompts,
    )

print(f"Queries n: {mrc_samples.shape[0]}")
print(f"From wiki n: {model_wiki_samples.shape[0]}")
print(f'Model: {model_name}')


query_embeddings = model.encode(mrc_samples['question'].tolist(), prompt_name="query", show_progress_bar=True)
document_embeddings = model.encode(model_wiki_samples['text'].tolist(), prompt_name="passage", show_progress_bar=True)
similarities = model.similarity(query_embeddings, document_embeddings) * 100

Sampled indices (Randomly Sampled 0~len(mrc)): [912, 204, 2253, 2006, 1828]
Raw wiki document ids in MRC samples df: [32341, 43424, 9619, 49578, 15171]
Extra Raw wiki document ids: [9144, 48269, 6717, 44352, 48544]
MRC samples df added: 5
Model-Wiki samples df added (Answer): 8
Model-Wiki samples df added (Extra): 7
Model-Wiki samples df added (Answer + Extra): 15
Queries n: 5
From wiki n: 15
Model: nlpai-lab/KoE5


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.64it/s]


In [77]:
import pandas as pd

# topn을 설정할 수 있게 인자로 받음
def create_topn_dataframe(mrc_samples, model_wiki_samples, raw_wiki, similarities, topn=3):
    # 각 query에 대해 topn 유사도를 가지는 context와 점수를 담을 리스트 준비
    topn_contexts = []

    for i, query_id in enumerate(mrc_samples['document_id']):
        # 유사도 기준으로 상위 n개의 index를 가져옴
        topn_indices = similarities[i].argsort()[-topn:][::-1]  # 큰 값부터 내림차순
        topn_scores = similarities[i][topn_indices]  # 해당 문서의 유사도 점수 가져옴
        
        # 새로운 row 생성
        row = {
            'id': mrc_samples.iloc[i]['id'],
            'question': mrc_samples.iloc[i]['question'],
            'answer': mrc_samples.iloc[i]['context']
        }
        
        # top1, top2, ..., topn context와 score 추가
        correct = False
        for j, idx in enumerate(topn_indices):
            doc_id = model_wiki_samples.iloc[idx]['document_id']
            
            # 해당 doc_id의 text를 raw_wiki에서 가져옴
            text = raw_wiki[raw_wiki['document_id'] == doc_id]['text'].values
            
            # 값이 존재하면 context 추가, 없으면 None
            row[f'top{j+1}_context'] = text[0] if len(text) > 0 else None
            row[f'top{j+1}_score'] = topn_scores[j]
            
            # 정답 여부 확인
            if doc_id == query_id:
                correct = True
        
        # correct 여부 추가
        row['correct'] = correct
        topn_contexts.append(row)

    # 새로운 DataFrame 생성
    df_topn = pd.DataFrame(topn_contexts)

    return df_topn

# 예시 실행
topn = 1  # 원하는 topn 설정
df_topn = create_topn_dataframe(mrc_samples, model_wiki_samples, raw_wiki, similarities, topn=topn)

# 결과 확인
df_topn.head(100)  # DataFrame의 상위 5개 행을 출력


,id,question,answer,top1_context,top1_score,correct
0,mrc-0-001427,이용순이 서재필의 시신을 뒷수습할 당시 서명원이 머무르던 국가는?,항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 ...,항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 ...,53.247135,True
1,mrc-0-002643,헤레로인이 독일기업이 예전에 나미비아에 큰 피해를 입힌 것에 대해 소송을 제기한 곳은?,1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1904년 수탈에 참다 ...,1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1904년 수탈에 참다 ...,63.065456,True
2,mrc-1-000170,호세 카레라스가 재단의 도움을 받아 병을 치료한 병원의 소재지는?,성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력같은 소식을 듣...,성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력같은 소식을 듣...,73.325851,True
3,mrc-0-000620,이만규가 최초로 선고받은 형벌은?,대법원 형사부 재판장으로 재직하던 1959년 10월 30일에 한강 둑 여인 살해 사...,대법원 형사부 재판장으로 재직하던 1959년 10월 30일에 한강 둑 여인 살해 사...,44.608517,True
4,mrc-0-004914,B형 항성보다 비이형 별의 스펙트럼에서 더욱 뚜렷하게 나타나는 현상은?,"Be형 항성(Be star), 껍질별(shell star) \n은 분광형 B의 항성...","Be형 항성(Be star), 껍질별(shell star) \n은 분광형 B의 항성...",70.490540,True


In [80]:
import pandas as pd

def create_mrc_input_csv(mrc_samples, model_wiki_samples, similarities, file_name="wiki_sp-topn_dn-topn.csv"):
    # N개의 top contexts를 담을 리스트 준비
    top_n = len(model_wiki_samples)

    # 각 query에 대해 topn 유사도를 가지는 context를 담을 새로운 DataFrame 생성
    topn_contexts = []

    for i, query_id in enumerate(mrc_samples['document_id']):
        # 유사도 기준으로 상위 n개의 index를 가져옴
        topn_indices = similarities[i].argsort()[-top_n:][::-1]  # 큰 값부터 내림차순
        topn_context = model_wiki_samples.iloc[topn_indices]['text'].tolist()  # context 추출

        # 새로운 row 생성
        row = {
            'id': mrc_samples.iloc[i]['id'],
            'question': mrc_samples.iloc[i]['question'],
        }

        # top1, top2, ..., topn context 추가
        for j in range(top_n):
            row[f'top{j+1}_context'] = topn_context[j]

        topn_contexts.append(row)

    # 새로운 DataFrame 생성
    result_to_mrc = pd.DataFrame(topn_contexts)

    # DataFrame을 .csv 파일로 저장
    result_to_mrc.to_csv(f"../data/pipeline/{file_name}", index=False)

    # 결과 반환
    return result_to_mrc

# 함수 호출 예시
mrc_input_df = create_mrc_input_csv(mrc_samples, model_wiki_samples, similarities, file_name=f"wiki_sp-topn_dn-topn.csv")

# 결과 확인
mrc_input_df.head(2)


,id,question,top1_context,top2_context,top3_context,top4_context,top5_context,top6_context,top7_context,top8_context,top9_context,top10_context,top11_context,top12_context,top13_context,top14_context,top15_context
0,mrc-0-001427,이용순이 서재필의 시신을 뒷수습할 당시 서명원이 머무르던 국가는?,"이는 갑신정변으로 서재필의 형제와 조카, 이복 형제들이 몰살당하고 살아남은 조카들도...",이용순 (대한제국) 항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대...,오필선 대법원 형사부 재판장으로 재직하던 1959년 10월 30일에 한강 둑 여인 ...,헤레로·나마 집단학살 1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1...,"이렇듯 정치에서 실패를 겪은 이후, 그레고리오 4세는 자신의 남은 임기의 대부분을 ...","단지, 나미비아에 대해 1884년부터 1915년까지의 식민통치만을 공식 인정하고, ...","경주 전 민애왕릉 신라 제44대 민애왕(838∼839)의 무덤으로 높이 3.8m, ...",바보회 한국의 근로기준법은 1953년 5월 10일에 제정되었다. 근로기준법이 제정된...,"치앙라이 유나이티드 FC 2016년 JARKEN 그룹과 협약을 맺으며, 구단의 브랜...",그러나 대다수 프랑크 주교들은 이러한 교황의 주장에 대해서는 아랑곳 하지 않았다. ...,교황 그레고리오 4세 그러나 세월이 흐르면서 프랑크 황제에 대한 교황의 의존성은 루...,055형 구축함 (전투기 목표물 기준의 탐지거리) * 중국 - 055형 구축함 34...,호세 카레라스 성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력...,"Be형 항성 Be형 항성(Be star), 껍질별(shell star) 은 분광형...","그러나 질량방출은 빠른 회전 속도만으로는 발생하지 않을 확률이 크며, 여기에 자기장..."
1,mrc-0-002643,헤레로인이 독일기업이 예전에 나미비아에 큰 피해를 입힌 것에 대해 소송을 제기한 곳은?,헤레로·나마 집단학살 1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1...,"단지, 나미비아에 대해 1884년부터 1915년까지의 식민통치만을 공식 인정하고, ...",그러나 대다수 프랑크 주교들은 이러한 교황의 주장에 대해서는 아랑곳 하지 않았다. ...,"이렇듯 정치에서 실패를 겪은 이후, 그레고리오 4세는 자신의 남은 임기의 대부분을 ...",호세 카레라스 성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력...,교황 그레고리오 4세 그러나 세월이 흐르면서 프랑크 황제에 대한 교황의 의존성은 루...,"치앙라이 유나이티드 FC 2016년 JARKEN 그룹과 협약을 맺으며, 구단의 브랜...",오필선 대법원 형사부 재판장으로 재직하던 1959년 10월 30일에 한강 둑 여인 ...,바보회 한국의 근로기준법은 1953년 5월 10일에 제정되었다. 근로기준법이 제정된...,"이는 갑신정변으로 서재필의 형제와 조카, 이복 형제들이 몰살당하고 살아남은 조카들도...",055형 구축함 (전투기 목표물 기준의 탐지거리) * 중국 - 055형 구축함 34...,이용순 (대한제국) 항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대...,"경주 전 민애왕릉 신라 제44대 민애왕(838∼839)의 무덤으로 높이 3.8m, ...","Be형 항성 Be형 항성(Be star), 껍질별(shell star) 은 분광형...","그러나 질량방출은 빠른 회전 속도만으로는 발생하지 않을 확률이 크며, 여기에 자기장..."


In [2]:
import json
import pandas as pd
from datasets import load_from_disk
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer

import torch
from tqdm import tqdm
import numpy as np
import random

random.seed(42)


def create_topn_dataframe(mrc_samples, model_wiki_samples, raw_wiki, similarities, topn=3):
    # 각 query에 대해 topn 유사도를 가지는 context와 점수를 담을 리스트 준비
    topn_contexts = []

    for i, query_id in enumerate(mrc_samples['document_id']):
        # 유사도 기준으로 상위 n개의 index를 가져옴
        topn_scores, topn_indices = torch.topk(similarities[i], k=topn, largest=True)
        
        # 새로운 row 생성
        row = {
            'id': mrc_samples.iloc[i]['id'],
            'question': mrc_samples.iloc[i]['question'],
            'answer': mrc_samples.iloc[i]['context']
        }
        
        # top1, top2, ..., topn context와 score 추가
        correct = False
        for j, idx in enumerate(topn_indices):
            doc_id = model_wiki_samples.iloc[idx]['document_id']
            
            # 해당 doc_id의 text를 raw_wiki에서 가져옴
            text = raw_wiki[raw_wiki['document_id'] == doc_id]['text'].values
            
            # 값이 존재하면 context 추가, 없으면 None
            row[f'top{j+1}_context'] = text[0] if len(text) > 0 else None
            row[f'top{j+1}_score'] = topn_scores[j]
            
            # 정답 여부 확인
            if doc_id == query_id:
                correct = True
        
        # correct 여부 추가
        row['correct'] = correct
        topn_contexts.append(row)

    # 새로운 DataFrame 생성
    df_topn = pd.DataFrame(topn_contexts)

    return df_topn


def create_mrc_input_csv(mrc_samples, model_wiki_samples, similarities, file_name="wiki_sp-topn_dn-topn.csv"):
    # N개의 top contexts를 담을 리스트 준비
    top_n = len(model_wiki_samples)

    # 각 query에 대해 topn 유사도를 가지는 context를 담을 새로운 DataFrame 생성
    topn_contexts = []

    for i, query_id in enumerate(mrc_samples['document_id']):
        # 유사도 기준으로 상위 n개의 index를 가져옴
        topn_indices = similarities[i].argsort()[-top_n:][::-1]  # 큰 값부터 내림차순
        topn_context = model_wiki_samples.iloc[topn_indices]['text'].tolist()  # context 추출

        # 새로운 row 생성
        row = {
            'id': mrc_samples.iloc[i]['id'],
            'question': mrc_samples.iloc[i]['question'],
        }

        # top1, top2, ..., topn context 추가
        for j in range(top_n):
            row[f'top{j+1}_context'] = topn_context[j]

        topn_contexts.append(row)

    # 새로운 DataFrame 생성
    result_to_mrc = pd.DataFrame(topn_contexts)

    # DataFrame을 .csv 파일로 저장
    result_to_mrc.to_csv(f"../data/pipeline/{file_name}", index=False)

    # 결과 반환
    return result_to_mrc


def calculate_topn_percentage(mrc_samples, model_wiki_samples, similarities, topn):
    topn_percentages = []

    for i in range(len(mrc_samples)):
        total_score = np.sum(similarities[i])  # 전체 유사도 합
        
        # 각 topn에 대해 유사도를 계산
        topn_scores = []
        for n in topn:
            topn_indices = similarities[i].argsort()[-n:][::-1]  # 상위 n개의 index
            topn_sum = np.sum(similarities[i][topn_indices])  # 상위 n개의 유사도 합
            percentage = (topn_sum / total_score) * 100  # 퍼센트로 변환
            topn_scores.append(percentage)
        
        topn_percentages.append(topn_scores)

    return topn_percentages


'''data'''
# data = json.load(open('data/raw/wikipedia_documents.json'))
model_data = json.load(open('../data/preprocessed/wikipedia_model.json'))
model_wiki = pd.DataFrame(model_data).T
raw_data = json.load(open('../data/raw/wikipedia_documents.json'))
raw_wiki = pd.DataFrame(raw_data).T

dataset = load_from_disk("../data/raw/train_dataset/")
train_df = pd.DataFrame(dataset['train'])
valid_df = pd.DataFrame(dataset['validation'])
mrc = pd.concat([train_df, valid_df])

# query_n = 50 # query number
# wiki_n = 100 # wiki number
query_n = 5 # query number
wiki_n = 10 # wiki number


assert query_n <= wiki_n, 'query_n should be smaller than total wiki_n'

sampled_indices = random.sample(range(len(mrc)), query_n)
print(f'Sampled indices (Randomly Sampled 0~len(mrc)): {sampled_indices}')

mrc_samples = mrc.iloc[sampled_indices]
raw_wiki_document_ids = mrc_samples['document_id'].tolist()
model_wiki_samples = model_wiki[model_wiki['document_id'].isin(raw_wiki_document_ids)]
available_document_ids = [i for i in range(model_wiki['document_id'].max()) if i not in raw_wiki_document_ids]
extra_raw_wiki_document_ids = random.sample(available_document_ids, wiki_n - query_n)
print(f'Raw wiki document ids in MRC samples df: {raw_wiki_document_ids}')
print(f'Extra Raw wiki document ids: {extra_raw_wiki_document_ids}')
print(f'MRC samples df added: {mrc_samples.shape[0]}')
print(f'Model-Wiki samples df added (Answer): {model_wiki_samples.shape[0]}')

extra_model_wiki_samples = model_wiki[model_wiki['document_id'].isin(extra_raw_wiki_document_ids)]
model_wiki_samples = pd.concat([model_wiki_samples, extra_model_wiki_samples])
print(f'Model-Wiki samples df added (Extra): {extra_model_wiki_samples.shape[0]}')
print(f'Model-Wiki samples df added (Answer + Extra): {model_wiki_samples.shape[0]}')


'''model'''
prompts = {
    "query": "query: ",        # 검색 쿼리 프롬프트
    "passage": "passage: "     # 문서 패시지 프롬프트
}

model_name = "nlpai-lab/KoE5"
model = SentenceTransformer(
    model_name_or_path=model_name, 
    device='cuda', 
    similarity_fn_name='dot',
    # truncate_dim=512,
    # model_kwargs={"torch_dtype": torch.bfloat16},
    prompts=prompts,
    )

print(f"Queries n: {mrc_samples.shape[0]}")
print(f"From wiki n: {model_wiki_samples.shape[0]}")
print(f'Model: {model_name}')


query_embeddings = model.encode(mrc_samples['question'].tolist(), prompt_name="query", show_progress_bar=True)
document_embeddings = model.encode(model_wiki_samples['text'].tolist(), prompt_name="passage", show_progress_bar=True)
similarities = model.similarity(query_embeddings, document_embeddings) * 100

similarities


Sampled indices (Randomly Sampled 0~len(mrc)): [912, 204, 2253, 2006, 1828]
Raw wiki document ids in MRC samples df: [32341, 43424, 9619, 49578, 15171]
Extra Raw wiki document ids: [9144, 48269, 6717, 44352, 48544]
MRC samples df added: 5
Model-Wiki samples df added (Answer): 8
Model-Wiki samples df added (Extra): 7
Model-Wiki samples df added (Answer + Extra): 15
Queries n: 5
From wiki n: 15
Model: nlpai-lab/KoE5


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.64it/s]


tensor([[-2.2337e+00, -4.7674e+00, -1.0738e+01,  5.0292e+01,  5.3247e+01,
          1.0532e+01,  6.5705e+00,  1.9840e+01,  5.9035e-01,  1.4323e+00,
          7.3541e+00,  2.1893e+00,  4.9346e+00,  1.6994e+00, -8.1662e-02],
        [ 1.5790e+01, -4.2117e+00, -1.2475e+01, -2.5316e+00,  3.7956e+00,
          6.3065e+01,  4.4784e+01,  6.3577e+00,  1.3118e+01,  1.8448e+01,
          1.6735e+01,  5.2417e+00, -3.3768e+00,  1.1982e+01, -1.2563e+00],
        [ 7.3326e+01,  1.7774e-01, -5.8971e+00,  6.1544e+00,  3.0184e+00,
          1.1316e+01,  3.4152e+00, -2.2728e+00,  8.3321e+00,  9.9440e+00,
          1.9132e+01, -1.3487e+00,  3.3848e+00,  1.2768e+01, -1.2657e+00],
        [ 5.8597e+00,  4.7890e-01, -5.9964e+00,  1.9110e+01,  1.4864e+01,
          1.3169e+01,  1.6136e+01,  4.4609e+01, -7.2318e+00, -2.0689e+00,
          1.6804e+00,  1.0535e+01,  6.9888e+00,  6.4817e-02, -5.9695e+00],
        [-1.4320e+00,  7.0491e+01,  6.0951e+01,  4.4773e-01,  1.9061e+00,
         -6.6069e+00, -4.4933e+00,

In [7]:
def create_mrc_input_csv(mrc_samples, model_wiki_samples, similarities, file_name="wiki_sp-topn_dn-topn.csv"):
    # N개의 top contexts를 담을 리스트 준비
    top_n = len(model_wiki_samples)
    print(top_n)

    # 각 query에 대해 topn 유사도를 가지는 context를 담을 새로운 DataFrame 생성
    topn_contexts = []

    for i, query_id in enumerate(mrc_samples['document_id']):
        # 유사도 기준으로 상위 n개의 index를 가져옴
        topn_values, topn_indices = torch.topk(similarities[i], k=top_n, largest=True)
        topn_context = model_wiki_samples.iloc[topn_indices.cpu().numpy()]['text'].tolist()


        # 새로운 row 생성
        row = {
            'id': mrc_samples.iloc[i]['id'],
            'question': mrc_samples.iloc[i]['question'],
        }

        # top1, top2, ..., topn context 추가
        for j in range(top_n):
            row[f'top{j+1}_context'] = topn_context[j]

        topn_contexts.append(row)

    # 새로운 DataFrame 생성
    result_to_mrc = pd.DataFrame(topn_contexts)

    # DataFrame을 .csv 파일로 저장
    result_to_mrc.to_csv(f"../data/pipeline/{file_name}", index=False)

    # 결과 반환
    return result_to_mrc


create_mrc_input_csv(mrc_samples, model_wiki_samples, similarities, file_name=f"wiki_sp-topn_dn-topn.csv")

15


,id,question,top1_context,top2_context,top3_context,top4_context,top5_context,top6_context,top7_context,top8_context,top9_context,top10_context,top11_context,top12_context,top13_context,top14_context,top15_context
0,mrc-0-001427,이용순이 서재필의 시신을 뒷수습할 당시 서명원이 머무르던 국가는?,"이는 갑신정변으로 서재필의 형제와 조카, 이복 형제들이 몰살당하고 살아남은 조카들도...",이용순 (대한제국) 항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대...,오필선 대법원 형사부 재판장으로 재직하던 1959년 10월 30일에 한강 둑 여인 ...,헤레로·나마 집단학살 1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1...,"이렇듯 정치에서 실패를 겪은 이후, 그레고리오 4세는 자신의 남은 임기의 대부분을 ...","단지, 나미비아에 대해 1884년부터 1915년까지의 식민통치만을 공식 인정하고, ...","경주 전 민애왕릉 신라 제44대 민애왕(838∼839)의 무덤으로 높이 3.8m, ...",바보회 한국의 근로기준법은 1953년 5월 10일에 제정되었다. 근로기준법이 제정된...,"치앙라이 유나이티드 FC 2016년 JARKEN 그룹과 협약을 맺으며, 구단의 브랜...",그러나 대다수 프랑크 주교들은 이러한 교황의 주장에 대해서는 아랑곳 하지 않았다. ...,교황 그레고리오 4세 그러나 세월이 흐르면서 프랑크 황제에 대한 교황의 의존성은 루...,055형 구축함 (전투기 목표물 기준의 탐지거리) * 중국 - 055형 구축함 34...,호세 카레라스 성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력...,"Be형 항성 Be형 항성(Be star), 껍질별(shell star) 은 분광형...","그러나 질량방출은 빠른 회전 속도만으로는 발생하지 않을 확률이 크며, 여기에 자기장..."
1,mrc-0-002643,헤레로인이 독일기업이 예전에 나미비아에 큰 피해를 입힌 것에 대해 소송을 제기한 곳은?,헤레로·나마 집단학살 1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1...,"단지, 나미비아에 대해 1884년부터 1915년까지의 식민통치만을 공식 인정하고, ...",그러나 대다수 프랑크 주교들은 이러한 교황의 주장에 대해서는 아랑곳 하지 않았다. ...,"이렇듯 정치에서 실패를 겪은 이후, 그레고리오 4세는 자신의 남은 임기의 대부분을 ...",호세 카레라스 성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력...,교황 그레고리오 4세 그러나 세월이 흐르면서 프랑크 황제에 대한 교황의 의존성은 루...,"치앙라이 유나이티드 FC 2016년 JARKEN 그룹과 협약을 맺으며, 구단의 브랜...",오필선 대법원 형사부 재판장으로 재직하던 1959년 10월 30일에 한강 둑 여인 ...,바보회 한국의 근로기준법은 1953년 5월 10일에 제정되었다. 근로기준법이 제정된...,"이는 갑신정변으로 서재필의 형제와 조카, 이복 형제들이 몰살당하고 살아남은 조카들도...",055형 구축함 (전투기 목표물 기준의 탐지거리) * 중국 - 055형 구축함 34...,이용순 (대한제국) 항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대...,"경주 전 민애왕릉 신라 제44대 민애왕(838∼839)의 무덤으로 높이 3.8m, ...","Be형 항성 Be형 항성(Be star), 껍질별(shell star) 은 분광형...","그러나 질량방출은 빠른 회전 속도만으로는 발생하지 않을 확률이 크며, 여기에 자기장..."
2,mrc-1-000170,호세 카레라스가 재단의 도움을 받아 병을 치료한 병원의 소재지는?,호세 카레라스 성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력...,"이렇듯 정치에서 실패를 겪은 이후, 그레고리오 4세는 자신의 남은 임기의 대부분을 ...","치앙라이 유나이티드 FC 2016년 JARKEN 그룹과 협약을 맺으며, 구단의 브랜...",헤레로·나마 집단학살 1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1...,그러나 대다수 프랑크 주교들은 이러한 교황의 주장에 대해서는 아랑곳 하지 않았다. ...,교황 그레고리오 4세 그러나 세월이 흐르면서 프랑크 황제에 대한 교황의 의존성은 루...,이용순 (대한제국) 항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대...,"단지, 나미비아에 대해 1884년부터 1915년까지의 식민통치만을 공식 인정하고, ...","경주 전 민애왕릉 신라 제44대 민애왕(838∼839)의 무덤으로 높이 3.8m, ...","이는 갑신정변으로 서재필의 형제와 조카, 이복 형제들이 몰살당하고 살아남은 조카들도...","Be형 항성 Be형 항성(Be star), 껍질별(shell star) 은 분광형...",055형 구축함 (전투기 목표물 기준의 탐지거리) * 중국 - 055형 구축함 34...,바보회 한국의 근로기준법은 1953년 5월 10일에 제정되었다. 근로기준법이 제정된...,오필선 대법원 형사부 재판장으로 재직하던 1959년 10월 30일에 한강 둑 여인 ...,"그러나 질량방출은 빠른 회전 속도만으로는 발생하지 않을 확률이 크며, 여기에 자기장..."
3,mrc-0-000620,이만규가 최초로 선고받은 형벌은?,오필선 대법원 형사부 재판장으로 재직하던 1959년 10월 30일에 한강 둑 여인 ...,이용순 (대한제국) 항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대...,"단지, 나미비아에 대해 1884년부터 1915년까지의 식민통치만을 공식 인정하고, ...","이는 갑신정변으로 서재필의 형제와 조카, 이복 형제들이 몰살당하고 살아남은 조카들도...",헤레로·나마 집단학살 1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1...,바보회 한국의 근로기준법은 1953년 5월 10일에 제정되었다. 근로기준법이 제정된...,"경주 전 민애왕릉 신라 제44대 민애왕(838∼839)의 무덤으로 높이 3.8m, ...",호세 카레라스 성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력...,"이렇듯 정치에서 실패를 겪은 이후, 그레고리오 4세는 자신의 남은 임기의 대부분을 ...","Be형 항성 Be형 항성(Be star), 껍질별(shell star) 은 분광형...","치앙라이 유나이티드 FC 2016년 JARKEN 그룹과 협약을 맺으며, 구단의 브랜...",그러나 대다수 프랑크 주교들은 이러한 교황의 주장에 대해서는 아랑곳 하지 않았다. ...,055형 구축함 (전투기 목표물 기준의 탐지거리) * 중국 - 055형 구축함 34...,"그러나 질량방출은 빠른 회전 속도만으로는 발생하지 않을 확률이 크며, 여기에 자기장...",교황 그레고리오 4세 그러나 세월이 흐르면서 프랑크 황제에 대한 교황의 의존성은 루...
4,mrc-0-004914,B형 항성보다 비이형 별의 스펙트럼에서 더욱 뚜렷하게 나타나는 현상은?,"Be형 항성 Be형 항성(Be star), 껍질별(shell star) 은 분광형...","그러나 질량방출은 빠른 회전 속도만으로는 발생하지 않을 확률이 크며, 여기에 자기장...","이는 갑신정변으로 서재필의 형제와 조카, 이복 형제들이 몰살당하고 살아남은 조카들도...",055형 구축함 (전투기 목표물 기준의 탐지거리) * 중국 - 055형 구축함 34...,이용순 (대한제국) 항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대...,호세 카레라스 성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력...,그러나 대다수 프랑크 주교들은 이러한 교황의 주장에 대해서는 아랑곳 하지 않았다. ...,바보회 한국의 근로기준법은 1953년 5월 10일에 제정되었다. 근로기준법이 제정된...,교황 그레고리오 4세 그러나 세월이 흐르면서 프랑크 황제에 대한 교황의 의존성은 루...,"경주 전 민애왕릉 신라 제44대 민애왕

In [36]:
topn_scores, topn_indices = torch.topk(similarities[0], k=2, largest=True)
topn_scores.tolist()

[53.247135162353516, 50.2917366027832]

In [40]:
def create_topn_dataframe(mrc_samples, model_wiki_samples, raw_wiki, similarities, topn=3):
    # 각 query에 대해 topn 유사도를 가지는 context와 점수를 담을 리스트 준비
    topn_contexts = []

    for i, query_id in enumerate(mrc_samples['document_id']):
        # 유사도 기준으로 상위 n개의 index를 가져옴
        topn_scores, topn_indices = torch.topk(similarities[i], k=topn, largest=True)
        if topn > 1:
            topn_scores, topn_indices = topn_scores.tolist(), topn_indices.tolist()
        else:
            topn_scores, topn_indices = top_scores.item(), top_indices.item()
        
        # 새로운 row 생성
        row = {
            'id': mrc_samples.iloc[i]['id'],
            'question': mrc_samples.iloc[i]['question'],
            'answer': mrc_samples.iloc[i]['context']
        }
        
        # top1, top2, ..., topn context와 score 추가
        correct = False
        for j, idx in enumerate(topn_indices):
            doc_id = model_wiki_samples.iloc[idx]['document_id']
            
            # 해당 doc_id의 text를 raw_wiki에서 가져옴
            text = raw_wiki[raw_wiki['document_id'] == doc_id]['text'].values
            
            # 값이 존재하면 context 추가, 없으면 None
            row[f'top{j+1}_context'] = text[0] if len(text) > 0 else None
            row[f'top{j+1}_score'] = topn_scores[j]
            
            # 정답 여부 확인
            if doc_id == query_id:
                correct = True
        
        # correct 여부 추가
        row['correct'] = correct
        topn_contexts.append(row)

    # 새로운 DataFrame 생성
    df_topn = pd.DataFrame(topn_contexts)

    return df_topn

df_topn = create_topn_dataframe(mrc_samples, model_wiki_samples, raw_wiki, similarities, topn=3)
df_topn.head(100)

,id,question,answer,top1_context,top1_score,top2_context,top2_score,top3_context,top3_score,correct
0,mrc-0-001427,이용순이 서재필의 시신을 뒷수습할 당시 서명원이 머무르던 국가는?,항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 ...,항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 ...,53.247135,항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 ...,50.291737,대법원 형사부 재판장으로 재직하던 1959년 10월 30일에 한강 둑 여인 살해 사...,19.839565,True
1,mrc-0-002643,헤레로인이 독일기업이 예전에 나미비아에 큰 피해를 입힌 것에 대해 소송을 제기한 곳은?,1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1904년 수탈에 참다 ...,1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1904년 수탈에 참다 ...,63.065456,1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1904년 수탈에 참다 ...,44.783939,그러나 세월이 흐르면서 프랑크 황제에 대한 교황의 의존성은 루트비히 1세와 그의 세...,18.447975,True
2,mrc-1-000170,호세 카레라스가 재단의 도움을 받아 병을 치료한 병원의 소재지는?,성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력같은 소식을 듣...,성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력같은 소식을 듣...,73.325851,그러나 세월이 흐르면서 프랑크 황제에 대한 교황의 의존성은 루트비히 1세와 그의 세...,19.132093,"2016년 JARKEN 그룹과 협약을 맺으며, 구단의 브랜드 강화 및 마케팅 전술 ...",12.768114,True
3,mrc-0-000620,이만규가 최초로 선고받은 형벌은?,대법원 형사부 재판장으로 재직하던 1959년 10월 30일에 한강 둑 여인 살해 사...,대법원 형사부 재판장으로 재직하던 1959년 10월 30일에 한강 둑 여인 살해 사...,44.608517,항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 ...,19.110022,1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1904년 수탈에 참다 ...,16.135895,True
4,mrc-0-004914,B형 항성보다 비이형 별의 스펙트럼에서 더욱 뚜렷하게 나타나는 현상은?,"Be형 항성(Be star), 껍질별(shell star) \n은 분광형 B의 항성...","Be형 항성(Be star), 껍질별(shell star) \n은 분광형 B의 항성...",70.490540,"Be형 항성(Be star), 껍질별(shell star) \n은 분광형 B의 항성...",60.951488,항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 ...,1.906148,True


In [43]:
df_topn

,id,question,answer,top1_context,top1_score,top2_context,top2_score,top3_context,top3_score,correct
0,mrc-0-001427,이용순이 서재필의 시신을 뒷수습할 당시 서명원이 머무르던 국가는?,항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 ...,항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 ...,53.247135,항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 ...,50.291737,대법원 형사부 재판장으로 재직하던 1959년 10월 30일에 한강 둑 여인 살해 사...,19.839565,True
1,mrc-0-002643,헤레로인이 독일기업이 예전에 나미비아에 큰 피해를 입힌 것에 대해 소송을 제기한 곳은?,1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1904년 수탈에 참다 ...,1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1904년 수탈에 참다 ...,63.065456,1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1904년 수탈에 참다 ...,44.783939,그러나 세월이 흐르면서 프랑크 황제에 대한 교황의 의존성은 루트비히 1세와 그의 세...,18.447975,True
2,mrc-1-000170,호세 카레라스가 재단의 도움을 받아 병을 치료한 병원의 소재지는?,성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력같은 소식을 듣...,성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력같은 소식을 듣...,73.325851,그러나 세월이 흐르면서 프랑크 황제에 대한 교황의 의존성은 루트비히 1세와 그의 세...,19.132093,"2016년 JARKEN 그룹과 협약을 맺으며, 구단의 브랜드 강화 및 마케팅 전술 ...",12.768114,True
3,mrc-0-000620,이만규가 최초로 선고받은 형벌은?,대법원 형사부 재판장으로 재직하던 1959년 10월 30일에 한강 둑 여인 살해 사...,대법원 형사부 재판장으로 재직하던 1959년 10월 30일에 한강 둑 여인 살해 사...,44.608517,항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 ...,19.110022,1884년 나마쿠아(현재의 나미비아)를 식민지화 한 독일은 1904년 수탈에 참다 ...,16.135895,True
4,mrc-0-004914,B형 항성보다 비이형 별의 스펙트럼에서 더욱 뚜렷하게 나타나는 현상은?,"Be형 항성(Be star), 껍질별(shell star) \n은 분광형 B의 항성...","Be형 항성(Be star), 껍질별(shell star) \n은 분광형 B의 항성...",70.490540,"Be형 항성(Be star), 껍질별(shell star) \n은 분광형 B의 항성...",60.951488,항일의병장인 이교문 (李敎文)의 손자이자 이일의 아들이다. 5대조 이유원이 보성군 ...,1.906148,True


In [45]:
def calculate_topn_percentage(df_topn, topn_list):
    # 각 topn에 대한 정답 비율을 저장할 딕셔너리
    topn_correct_percentage = {}

    # topn 리스트에 있는 각 값을 처리
    for topn in topn_list:
        correct_count = 0
        total_count = len(df_topn)

        # 각 row에 대해 정답이 있는지 확인
        for index, row in df_topn.iterrows():
            # topn에 해당하는 열을 검사 (top1, top2, ..., topn)
            correct_found = False
            for n in range(1, topn + 1):
                if row[f'top{n}_context'] and row['correct']:  # correct가 True이면 정답
                    correct_found = True
                    break

            if correct_found:
                correct_count += 1

        # 정답 비율 계산
        percentage = (correct_count / total_count) * 100
        topn_correct_percentage[f'top{topn}'] = percentage

    return topn_correct_percentage


# 예시 실행
topn_list = [i for i in range(mrc_samples.shape[0])]  # 원하는 topn 설정
topn_percentage = calculate_topn_percentage(df_topn, topn_list)

# 결과 출력
for topn, percentage in topn_percentage.items():
    print(f"{topn}: {percentage:.2f}% correct")


top0: 0.00% correct
top1: 100.00% correct
top2: 100.00% correct
top3: 100.00% correct
top4: 100.00% correct


In [47]:
sorted_scores, sorted_indices = torch.sort(similarities[0], descending=True)
sorted_scores.tolist(), sorted_indices.tolist()

([53.247135162353516,
  50.2917366027832,
  19.83956527709961,
  10.53150463104248,
  7.354137897491455,
  6.570520877838135,
  4.9345622062683105,
  2.189324140548706,
  1.699363112449646,
  1.4323310852050781,
  0.5903500914573669,
  -0.08166227489709854,
  -2.2337265014648438,
  -4.76735258102417,
  -10.737503051757812],
 [4, 3, 7, 5, 10, 6, 12, 11, 13, 9, 8, 14, 0, 1, 2])

In [53]:
model_wiki_dict = model_wiki_samples.set_index('document_id')['text'].to_dict()
model_wiki_dict

{9619: "호세 카레라스 성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력같은 소식을 듣는다. 바로 급성 백혈병 진단을 받은 것이다. 그러나 그는 생사를 오가는 투병 끝에 결국 기적적으로 완치가 됐고, 무대에 복귀할 수 있게 된다. 복귀 이후 급속도로 기량이 쇠퇴하게 되어 많은 사람들에게 안타까움을 샀지만, 1988년 '호세 카레라스 백혈병 재단'을 설립하여 백혈병 환자들의 치료비를 지원하고 수많은 자선 공연에서 노래를 부르는 등 사회 공헌에 전념하고 있다.  한편 카탈루냐 출신인 호세 카레라스와 카스티야 출신인 플라시도 도밍고는 라이벌인 동시에 지역 감정, 그리고 스페인의 정치적 상황으로 인해 사이가 좋지 못했다. 백혈병 투병을 하게 된 카레라스는 계속된 항암 치료와 골수 이식으로 경제적 어려움을 겪는다. 그 때 카레라스는 마드리드에 위치한 에르모사 백혈병 재단이 운영하는 병원이 있다는 것을 알게 되고 무료로 치료를 받아 건강을 회복한다. 완치판정을 받은 카레라스는 에르모사 재단에 감사 표하는 마음으로 후원 회원이 되기 위한 절차를 밟는데, 알고보니 에르모사 재단은 플라시도 도밍고가 설립한 재단이었다. 도밍고가 이 재단을 설립한 취지는 호세 카레라스의 병을 치료해주기 위한 것이었다. 라이벌인 카레라스의 자존심이 상하지 않도록 익명으로 도움을 주려했던 것. 카레라스는 도밍고의 우정에 감동했고 그 이후로 둘은 절친한 친구가 된다. 카레라스가 백혈병 재단을 설립하게 된 것도 이것이 계기가 되었다.",
 15171: '그러나 질량방출은 빠른 회전 속도만으로는 발생하지 않을 확률이 크며, 여기에 자기장 및 비복사 항성맥동 등의 추가적인 방출 메카니즘이 더해져야 한다. 비이형 별이 잠깐 동안만 존재하는 상태임을 고려하면 항성맥동이 더 관련이 있을 것으로 보이지만, 이 분야의 연구는 아직 진행 중이다.  비이형 별들은 보통 변광성이며, 항성주위 가스원반이 오래 존재하지 못하고, 원반이 항성의 빛을 분산하는 특징에 기반하여 카시오페이아자리 감마형 변

In [54]:
model_wiki_samples.head(2)

,text,corpus_source,url,domain,title,author,html,document_id
13907,호세 카레라스 성악가로서 한창 전성기를 구가하던 1987년 호세 카레라스는 청천벽력...,위키피디아,None,None,호세 카레라스,None,None,9619
21203,"Be형 항성 Be형 항성(Be star), 껍질별(shell star) 은 분광형...",위키피디아,None,None,Be형 항성,None,None,15171
